# Section: Encrypted Deep Learning

- Lesson: Reviewing Additive Secret Sharing
- Lesson: Encrypted Subtraction and Public/Scalar Multiplication
- Lesson: Encrypted Computation in PySyft
- Project: Build an Encrypted Database
- Lesson: Encrypted Deep Learning in PyTorch
- Lesson: Encrypted Deep Learning in Keras
- Final Project

# Lesson: Reviewing Additive Secret Sharing

_For more great information about SMPC protocols like this one, visit https://mortendahl.github.io. With permission, Morten's work directly inspired this first teaching segment._

In [1]:
!pip install --upgrade syft

Requirement already up-to-date: syft in /usr/local/lib/python3.6/dist-packages (0.1.21a1)


In [0]:
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [0]:
x = encrypt(encode(5.5))
x

[76415910642592650282410530616961883967,
 29136920984719121970593682754902659143,
 188420513847855474817441064409051201303]

In [0]:
y = encrypt(encode(2.3))
y

[166195622956431362823577447161467892485,
 246469003377878055249095104209355375995,
 175282064616025076068218004190138220345]

In [0]:
z = add(x,y)
z

(242611533599024013105987977778429776452,
 275605924362597177219688786964258035138,
 69729232988713303815213790818823677235)

In [0]:
decode(decrypt(z))

7.79999999

# Lesson: Encrypted Subtraction and Public/Scalar Multiplication

In [0]:
field = 23740629843760239486723

In [0]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [0]:
(bob_x_share + alices_x_share) % field

5

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [0]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [0]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [0]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [0]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [0]:
(bob_z_share + alice_z_share) % field

4

In [0]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [0]:
bob_x_share + alice_x_share

15

In [0]:
bob_y_share + alice_y_share

11

In [0]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [0]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [0]:
x = encrypt(encode(5.5))
x

[3540224615689186850666997178009893611,
 231634467016271076990970851752039249528,
 58798653843206983228807428850866601274]

In [0]:
z = imul(x, 3)

In [0]:
decode(decrypt(z))

16.5

# Lesson: Encrypted Computation in PySyft

In [3]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0715 00:23:21.999238 139681796171648 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0715 00:23:22.017191 139681796171648 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [0]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [0]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [29]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [30]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [32]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [33]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [34]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [35]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [0]:
z = x + y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [31]:
z = x - y
z.get().float_precision()

AttributeError: ignored

In [0]:
z = x * y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [0]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [0]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [0]:
z = x == y
z.get().float_precision()

tensor([0., 0., 0., 0.])

# Project: Build an Encrypted Database

In [75]:
import string
bob.clear_objects()
alice.clear_objects()
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

In [0]:
char_index={}
index_char={}
for idx,char in enumerate(" "+string.ascii_letters+string.digits+string.punctuation):
  char_index[char]=idx
  index_char[idx]=char
  
  

In [0]:
def string_index(string_crt):
  indices=list()
  for char in string_crt:
    indices.append(char_index[char])

  return th.tensor(indices)

In [0]:
class string_key_value():
  def __init__(self,*owners):
    self.keys=list()
    self.values=list()
    self.owners=list(owners)
  def string_match(self,string_query):
    string_query=string_query.share(self.owners[0],self.owners[1],crypto_provider=self.owners[2])
    for idx,key in enumerate(self.keys):
      if len(string_query)==len(key):
        if (th.sum(key.sub(string_query))).get()==0:
          result=self.values[idx]
          result=result.get()
          return result
      

  def add_data(self,key,value):
    key=key.share(self.owners[0],self.owners[1],crypto_provider=self.owners[2])
    self.keys.append(key)
    value=value.share(self.owners[0],self.owners[1],crypto_provider=self.owners[2])

    self.values.append(value)


In [0]:
def index_string(index_crt):
  string_list=list()
  index_crt=list(index_crt.numpy())
  for i in index_crt:
    string_list.append(index_char[i])
  return "".join(string_list)

In [91]:
db=string_key_value(bob,alice,secure_worker)

<VirtualWorker id:bob #objects:14> <VirtualWorker id:alice #objects:18> <VirtualWorker id:secure_worker #objects:0>


In [0]:
db.add_data(string_index("key1"),string_index("value1"))

In [0]:
db.add_data(string_index("key2"),string_index("value2"))

In [0]:
match_key=string_index("key2")

In [0]:
result=db.string_match(match_key)

In [99]:
result_string=index_string(result)
result_string

'value2'

# Lesson: Encrypted Deep Learning in PyTorch

### Train a Model

In [6]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(2.9485)
tensor(9.6907)
tensor(10.5821)
tensor(0.9325)
tensor(0.8769)
tensor(0.8296)
tensor(0.7735)
tensor(0.6877)
tensor(0.5834)
tensor(0.4648)
tensor(0.3408)
tensor(0.2242)
tensor(0.1346)
tensor(0.0782)
tensor(0.0450)
tensor(0.0270)
tensor(0.0174)
tensor(0.0120)
tensor(0.0086)
tensor(0.0064)


In [7]:
model(data)

tensor([[ 0.0572],
        [-0.0323],
        [ 0.9775],
        [ 0.9994]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [0]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [9]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:99497974212 -> alice:36782629387]
 	-> (Wrapper)>[PointerTensor | me:45779793437 -> bob:64169565995]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:16690494985 -> alice:59671716694]
 	-> (Wrapper)>[PointerTensor | me:20210949258 -> bob:84651898958]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:96971646175 -> alice:13312025986]
 	-> (Wrapper)>[PointerTensor | me:66400910854 -> bob:9377596562]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:54458996411 -> alice:66153288266]
 	-> (Wrapper)>[PointerTensor | me:76209616124 -> bob:828489636

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [11]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:11305882392 -> alice:65818850167]
	-> (Wrapper)>[PointerTensor | me:43379962836 -> bob:44254893162]
	*crypto provider: secure_worker*

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [13]:
encrypted_prediction.get().float_precision()

tensor([[ 0.0580],
        [-0.0310],
        [ 0.9750],
        [ 0.9970]])

# Lesson: Encrypted Deep Learning in Keras


## Step 1: Public Training

Welcome to this tutorial! In the following notebooks you will learn how to provide private predictions. By private predictions, we mean that the data is constantly encrypted throughout the entire process. At no point is the user sharing raw data, only encrypted (that is, secret shared) data. In order to provide these private predictions, Syft Keras uses a library called [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) under the hood. TF Encrypted combines cutting-edge cryptographic and machine learning techniques, but you don't have to worry about this and can focus on your machine learning application.

You can start serving private predictions with only three steps:
- **Step 1**: train your model with normal Keras.
- **Step 2**: secure and serve your machine learning model (server).
- **Step 3**: query the secured model to receive private predictions (client). 

Alright, let's go through these three steps so you can deploy impactful machine learning services without sacrificing user privacy or model security.

Huge shoutout to the Dropout Labs ([@dropoutlabs](https://twitter.com/dropoutlabs)) and TF Encrypted ([@tf_encrypted](https://twitter.com/tf_encrypted)) teams for their great work which makes this demo possible, especially: Jason Mancuso ([@jvmancuso](https://twitter.com/jvmancuso)), Yann Dupis ([@YannDupis](https://twitter.com/YannDupis)), and Morten Dahl ([@mortendahlcs](https://github.com/mortendahlcs)). 

_Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials_

## Train Your Model in Keras

To use privacy-preserving machine learning techniques for your projects you should not have to learn a new machine learning framework. If you have basic [Keras](https://keras.io/) knowledge, you can start using these techniques with Syft Keras. If you have never used Keras before, you can learn a bit more about it through the [Keras documentation](https://keras.io). 

Before serving private predictions, the first step is to train your model with normal Keras. As an example, we will train a model to classify handwritten digits. To train this model we will use the canonical [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

We borrow [this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) from the reference Keras repository.  To train your classification model, you just run the cell below.

In [2]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0715 01:18:40.915485 140546383480704 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 69us/sample - loss: 2.3016 - acc: 0.1338 - val_loss: 2.3003 - val_acc: 0.1535
Epoch 2/2
60000/60000 [==============================] - 2s 34us/sample - loss: 2.2999 - acc: 0.1559 - val_loss: 2.2984 - val_acc: 0.1739
Test loss: 2.2983899822235108
Test accuracy: 0.1739


In [0]:
## Save your model's weights for future private prediction
model.save('short-conv-mnist.h5')

## Step 2: Load and Serve the Model

Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras.

To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run `hook = sy.KerasHook(tf.keras)`. This will add three important new methods to the Keras Sequential class:
 - `share`: will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
 - `serve`: this function will launch a serving queue, so that the TFEWorkers can can accept prediction requests on the secured model from external clients.
 - `shutdown_workers`: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker.

If you want learn more about MPC, you can read this excellent [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

W0715 01:18:57.476640 140546383480704 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0715 01:18:57.575459 140546383480704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



## Model

As you can see, we define almost the exact same model as before, except we provide a `batch_input_shape`. This allows TF Encrypted to better optimize the secure computations via predefined tensor shapes. For this MNIST demo, we'll send input data with the shape of (1, 28, 28, 1). 
We also return the logit instead of softmax because this operation is complex to perform using MPC, and we don't need it to serve prediction requests.

In [0]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

### Load Pre-trained Weights

With `load_weights` you can easily load the weights you have saved previously after training your model.

In [0]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

In [7]:
!ping localhost:4000
  

/bin/bash: ping: command not found


## Step 3: Setup Your Worker Connectors

Let's now connect to the TFEWorkers (`alice`, `bob`, and `carol`) required by TF Encrypted to perform private predictions. For each TFEWorker, you just have to specify a host.

These workers run a [TensorFlow server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server), which you can either manage manually (`AUTO = False`) or ask the workers to manage for you (`AUTO = True`). If choosing to manually manage them, you will be instructed to execute a terminal command on each worker's host device after calling `model.share()` below.  If all workers are hosted on a single device (e.g. `localhost`), you can choose to have Syft automatically manage the worker's TensorFlow server.

In [0]:
AUTO = False

alice = sy.TFEWorker(host='localhost:8887', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:8888', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:8889', auto_managed=AUTO)

## Step 4: Split the Model Into Shares

Thanks to `sy.KerasHook(tf.keras)` you can call the `share` method to transform your model into a TF Encrypted Keras model.

If you have asked to manually manage servers above then this step will not complete until they have all been launched. Note that your firewall may ask for Python to accept incoming connection.

In [0]:
model.share(alice, bob, carol)

W0715 01:20:30.971613 140546383480704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/keras/engine/base_layer_utils.py:29: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 01:20:30.973762 140546383480704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/tensor/native.py:403: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 01:20:30.981237 140546383480704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/tensor/native.py:101: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 01:20:31.027499 140546383480704 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/tensor/shared.py:62: calling extract_image_patches (from tensorflow.python.ops.array_ops) with ksizes is deprecated and will be removed in a future version.
Instructions fo

## Step 5: Launch 3 Servers

```
python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2```

In [0]:
!python -m tf_encrypted.player --config /tmp/tfe.config server0

In [11]:
!python -m tf_encrypted.player --config /tmp/tfe.config server1 

W0715 01:13:23.958401 140615319328640 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0715 01:13:23.974628 140615319328640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

I0715 01:13:24.014045 140615319328640 config.py:280] Creating server for 'server1' using ClusterSpec({'tfe': ['localhost:4000', 'localhost:4001', 'localhost:4002']})
W0715 01:13:24.014289 140615319328640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/config.py:281: The name tf.train.Server is deprecated. Please use tf.distribute.Server instead.

2019-07-15 01:13:24.018623: I tensorflow/core/platf

In [12]:
!python -m tf_encrypted.player --config /tmp/tfe.config server1 

KeyboardInterrupt: ignored

## Step 6: Serve the Model

Perfect! Now by calling `model.serve`, your model is ready to provide some private predictions. You can set `num_requests` to set a limit on the number of predictions requests served by the model; if not specified then the model will be served until interrupted.

In [0]:
model.serve(num_requests=3)

Served encrypted prediction 1 to client.
Served encrypted prediction 2 to client.
Served encrypted prediction 3 to client.


## Step 7: Run the Client

At this point open up and run the companion notebook: Section 4b - Encrytped Keras Client

## Step 8: Shutdown the Servers

Once your request limit above, the model will no longer be available for serving requests, but it's still secret shared between the three workers above. You can kill the workers by executing the cell below.

**Congratulations** on finishing Part 12: Secure Classification with Syft Keras and TFE!

In [0]:
model.shutdown_workers()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config /tmp/tfe.config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))

# Keystone Project - Mix and Match What You've Learned

Description: Take two of the concepts you've learned about in this course (Encrypted Computation, Federated Learning, Differential Privacy) and combine them for a use case of your own design. Extra credit if you can get your demo working with [WebSocketWorkers](https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials/advanced/websockets-example-MNIST) instead of VirtualWorkers! Then take your demo or example application, write a blogpost, and share that blogpost in #general-discussion on OpenMined's slack!!!

Inspiration:
- This Course's Code: https://github.com/Udacity/private-ai
- OpenMined's Tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials
- OpenMined's Blog: https://blog.openmined.org